In [1]:
from config import *
from dataloader import TextDataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, trange

from transformers import BertPreTrainedModel, BertModel
from transformers import AutoConfig, AutoTokenizer, BertConfig

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

In [2]:
df = pd.read_csv('../../data/frame2.csv')[['description_clear', 'salary_from_rate_and_gross_log']]

c:\Users\maksk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
import numpy as np
df['salary_from_rate_and_gross_log'] = np.exp(df['salary_from_rate_and_gross_log'])

In [4]:
df.reset_index(inplace=True)

In [5]:
class DescriptionDataset(Dataset):

    def __init__(self, data, maxlen, tokenizer): 
        #Store the contents of the file in a pandas dataframe
        self.df = data.reset_index()
        #Initialize the tokenizer for the desired transformer model
        self.tokenizer = tokenizer
        #Maximum length of the tokens list to keep all the sequences of fixed size
        self.maxlen = maxlen

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):    
        #Select the sentence and label at the specified index in the data frame
        excerpt = self.df.loc[index, 'description_clear']
        try:
            target = self.df.loc[index, 'salary_from_rate_and_gross_log']
        except:
            target = 0.0
        identifier = self.df.loc[index, 'index']
        #Preprocess the text to be suitable for the transformer
        tokens = self.tokenizer.tokenize(excerpt) 
        tokens = ['[CLS]'] + tokens + ['[SEP]'] 
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] 
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] 
        #Obtain the indices of the tokens in the BERT Vocabulary
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens) 
        input_ids = torch.tensor(input_ids) 
        #Obtain the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attention_mask = (input_ids != 0).long()
        
        target = torch.tensor(target, dtype=torch.float32)
        
        return input_ids, attention_mask, target

In [6]:
class BertRegresser(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        #The output layer that takes the [CLS] representation and gives an output
        self.cls_layer1 = nn.Linear(config.hidden_size,128)
        self.relu1 = nn.ReLU()
        self.ff1 = nn.Linear(128,128)
        self.tanh1 = nn.Tanh()
        self.ff2 = nn.Linear(128,1)

    def forward(self, input_ids, attention_mask):
        #Feed the input to Bert model to obtain contextualized representations
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #Obtain the representations of [CLS] heads
        logits = outputs.last_hidden_state[:,0,:]
        output = self.cls_layer1(logits)
        output = self.relu1(output)
        output = self.ff1(output)
        output = self.tanh1(output)
        output = self.ff2(output)
        return output, outputs

In [7]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs, device):
    best_acc = 0
    print('Start')
    for epoch in trange(epochs, desc="Epoch"):
        model.train()
        train_loss = 0
        train_rmse = 0
        for i, (input_ids, attention_mask, target) in enumerate(iterable=train_loader):
            optimizer.zero_grad()  
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            
            output, _ = model(input_ids=input_ids, attention_mask=attention_mask)
            target= torch.unsqueeze(target, dim=1)
            loss = criterion(output, target.type_as(output))
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_rmse += get_rmse(output, target.type_as(output), criterion=criterion)
            if i % 100 == 0:
                print(f'Batch:{i}/{len(train_loader)}')
        
        print(f"Training loss is {train_loss/len(train_loader)}\trmse is {train_rmse/len(train_loader)}")
        val_loss, val_rmse = evaluate(model=model, criterion=criterion, dataloader=val_loader, device=device)
        print("Epoch {} complete! Validation Loss : {}".format(epoch, val_loss))
        print(f"Epoch {epoch} complete! Validation loss is {val_loss}\trmse is {val_rmse}")

In [8]:
def evaluate(model, criterion, dataloader, device):
    model.eval()
    mean_acc, mean_loss, mean_err, count = 0, 0, 0, 0
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output, _ = model(input_ids, attention_mask)
            target= torch.unsqueeze(target, dim=1)
            mean_loss += criterion(output, target.type_as(output)).item()
            mean_err += get_rmse(output, target, criterion=criterion)
            count += 1
            
    return mean_loss/count, mean_err/count

In [9]:
from sklearn import metrics

def get_rmse(output, target, criterion):
    with torch.no_grad():
        err = torch.sqrt(criterion(target, output))
        return err

In [10]:
def predict(model, dataloader, device):
    predicted_label = []
    actual_label = []
    with torch.no_grad():
        for input_ids, attention_mask, target in (dataloader):
            
            input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
            output, _ = model(input_ids, attention_mask)
                        
            predicted_label += output
            actual_label += target
            
    return predicted_label

In [11]:
train_dataset, valid_dataset = train_test_split(df, test_size=0.3, random_state=0)
valid_dataset, test_dataset = train_test_split(valid_dataset, test_size=0.5, random_state=0)

In [12]:
config = BertConfig.from_pretrained("cointegrated/rubert-tiny", output_hidden_states=True)
## Tokenizer loaded from AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
## Creating the model from the desired transformer model
model = BertRegresser.from_pretrained("cointegrated/rubert-tiny", config=config)
## GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## Putting model to device
model = model.to(device)
## Takes as the input the logits of the positive class and computes the binary cross-entropy 
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
## Optimizer
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertRegresser: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertRegresser from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertRegresser from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertRegresser were not initialized from the model checkpoint at cointegrated/rubert-tiny

In [13]:
train_set = DescriptionDataset(data=train_dataset, maxlen=256, tokenizer=tokenizer)
valid_set = DescriptionDataset(data=valid_dataset, maxlen=256, tokenizer=tokenizer)
test_set = DescriptionDataset(data=test_dataset, maxlen=256, tokenizer=tokenizer)

In [14]:
train_loader = DataLoader(dataset=train_set, batch_size=50)
valid_loader = DataLoader(dataset=valid_set, batch_size=50)
test_loader = DataLoader(dataset=test_set, batch_size=50)

In [15]:
train(model=model, 
      criterion=criterion,
      optimizer=optimizer, 
      train_loader=train_loader,
      val_loader=valid_loader,
      epochs = 50,
     device = device)

Start


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors


Batch:0/2199
Batch:100/2199
Batch:200/2199
Batch:300/2199
Batch:400/2199
Batch:500/2199
Batch:600/2199
Batch:700/2199
Batch:800/2199
Batch:900/2199
Batch:1000/2199
Batch:1100/2199
Batch:1200/2199
Batch:1300/2199
Batch:1400/2199
Batch:1500/2199
Batch:1600/2199
Batch:1700/2199
Batch:1800/2199
Batch:1900/2199
Batch:2000/2199
Batch:2100/2199
Training loss is 5631816106.783083	rmse is 74741.2265625


Epoch:   2%|▏         | 1/50 [05:38<4:36:28, 338.53s/it]

Epoch 0 complete! Validation Loss : 5596397018.576271
Epoch 0 complete! Validation loss is 5596397018.576271	rmse is 74535.5390625
Batch:0/2199
Batch:100/2199
Batch:200/2199
Batch:300/2199
Batch:400/2199
Batch:500/2199
Batch:600/2199
Batch:700/2199
Batch:800/2199
Batch:900/2199
Batch:1000/2199
Batch:1100/2199
Batch:1200/2199
Batch:1300/2199
Batch:1400/2199
Batch:1500/2199
Batch:1600/2199
Batch:1700/2199
Batch:1800/2199
Batch:1900/2199
Batch:2000/2199
Batch:2100/2199
Training loss is 5594965657.087768	rmse is 74493.8203125


...

Epoch 47 complete! Validation Loss : 4047291531.3898306
Epoch 47 complete! Validation loss is 4047291531.3898306	rmse is 63279.796875
Batch:0/2199
Batch:100/2199
Batch:200/2199
Batch:300/2199
Batch:400/2199
Batch:500/2199
Batch:600/2199
Batch:700/2199
Batch:800/2199
Batch:900/2199
Batch:1000/2199
Batch:1100/2199
Batch:1200/2199
Batch:1300/2199
Batch:1400/2199
Batch:1500/2199
Batch:1600/2199
Batch:1700/2199
Batch:1800/2199
Batch:1900/2199
Batch:2000/2199
Batch:2100/2199
Training loss is 4047363556.8749433	rmse is 63239.71875


Epoch:  98%|█████████▊| 49/50 [4:43:13<05:54, 354.01s/it]

Epoch 48 complete! Validation Loss : 4018185019.9322033
Epoch 48 complete! Validation loss is 4018185019.9322033	rmse is 63048.9765625
Batch:0/2199
Batch:100/2199
Batch:200/2199
Batch:300/2199
Batch:400/2199
Batch:500/2199
Batch:600/2199
Batch:700/2199
Batch:800/2199
Batch:900/2199
Batch:1000/2199
Batch:1100/2199
Batch:1200/2199
Batch:1300/2199
Batch:1400/2199
Batch:1500/2199
Batch:1600/2199
Batch:1700/2199
Batch:1800/2199
Batch:1900/2199
Batch:2000/2199
Batch:2100/2199
Training loss is 4018288796.5220556	rmse is 63008.9375


Epoch: 100%|██████████| 50/50 [4:49:18<00:00, 347.16s/it]

Epoch 49 complete! Validation Loss : 3989239307.118644
Epoch 49 complete! Validation loss is 3989239307.118644	rmse is 62818.6796875


In [17]:
torch.save(model.state_dict(), 'regression_model/third.pth')

In [18]:
bert = BertRegresser.from_pretrained("cointegrated/rubert-tiny", config=config)
bert.load_state_dict(torch.load('regression_model/third.pth'))

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertRegresser: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertRegresser from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertRegresser from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertRegresser were not initialized from the model checkpoint at cointegrated/rubert-tiny

<All keys matched successfully>

In [74]:
train_set[0][1].unsqueeze(0)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [19]:
model.eval()
with torch.no_grad():
    output, z = model(train_set[0][0].to(device).unsqueeze(0), train_set[0][1].to(device).unsqueeze(0))
# for input_ids, attention_mask, target in (dataloader):
            
#             input_ids, attention_mask, target = input_ids.to(device), attention_mask.to(device), target.to(device)
#             output = model(input_ids, attention_mask)

In [20]:
z['last_hidden_state'][0][0]

tensor([ 1.0165, -0.5941,  0.8446, -1.0152, -1.1903,  0.8911, -1.0150,  0.6692,
        -0.0823,  1.0956,  1.3675, -0.9616, -0.6173,  1.1120,  0.2065,  0.7564,
         0.0050, -0.9943, -0.8755, -1.4005, -0.9810,  0.7830, -1.1040, -0.6733,
         1.1221, -0.4395,  1.2176,  0.8622,  1.1510,  0.9769, -0.9694,  1.0767,
        -0.6339,  0.9127, -0.0927, -0.8705,  0.8907, -1.0492,  1.1331, -0.2640,
        -1.3647,  0.4010,  1.4742, -1.0280, -0.9186,  0.5417, -0.6405,  0.9277,
        -1.1187, -1.3352,  1.1012, -1.1103,  0.9032,  0.9564, -1.3408,  1.0993,
         1.1243,  0.9001, -1.2059, -0.8907,  1.1102,  0.9942,  0.8966, -0.7747,
         1.0933,  1.4284, -1.1696, -0.8015,  0.7859, -0.9075, -1.1437, -0.9550,
        -1.3351, -0.9288,  0.9722,  1.1186,  1.0152,  0.4491,  1.1752, -0.5556,
         1.1111, -0.8800, -1.0476, -0.6195,  0.7390,  0.8985, -1.0188, -0.8994,
        -0.0845, -0.9677, -0.9305, -1.1115,  1.7626,  1.0709,  1.0571,  0.6062,
         1.2125,  1.1210, -1.0072,  1.06

In [22]:
bert.cuda()
bert.eval()

with torch.no_grad():
    output, z = bert(train_set[0][0].to(device).unsqueeze(0), train_set[0][1].to(device).unsqueeze(0))

In [23]:
z['last_hidden_state'][0][0]

tensor([ 1.0165, -0.5941,  0.8446, -1.0152, -1.1903,  0.8911, -1.0150,  0.6692,
        -0.0823,  1.0956,  1.3675, -0.9616, -0.6173,  1.1120,  0.2065,  0.7564,
         0.0050, -0.9943, -0.8755, -1.4005, -0.9810,  0.7830, -1.1040, -0.6733,
         1.1221, -0.4395,  1.2176,  0.8622,  1.1510,  0.9769, -0.9694,  1.0767,
        -0.6339,  0.9127, -0.0927, -0.8705,  0.8907, -1.0492,  1.1331, -0.2640,
        -1.3647,  0.4010,  1.4742, -1.0280, -0.9186,  0.5417, -0.6405,  0.9277,
        -1.1187, -1.3352,  1.1012, -1.1103,  0.9032,  0.9564, -1.3408,  1.0993,
         1.1243,  0.9001, -1.2059, -0.8907,  1.1102,  0.9942,  0.8966, -0.7747,
         1.0933,  1.4284, -1.1696, -0.8015,  0.7859, -0.9075, -1.1437, -0.9550,
        -1.3351, -0.9288,  0.9722,  1.1186,  1.0152,  0.4491,  1.1752, -0.5556,
         1.1111, -0.8800, -1.0476, -0.6195,  0.7390,  0.8985, -1.0188, -0.8994,
        -0.0845, -0.9677, -0.9305, -1.1115,  1.7626,  1.0709,  1.0571,  0.6062,
         1.2125,  1.1210, -1.0072,  1.06

In [22]:
torch.save(model, 'regression_model/second.pth')

In [6]:
import torch
model_new = torch.load('regression_model/second.pth', map_location='cuda')

PermissionError: [Errno 13] Permission denied: 'regression_model/checkpoint-22000'

In [26]:
model.eval()
with torch.no_grad():
    output, z = model(train_set[0][0].to(device).unsqueeze(0), train_set[0][1].to(device).unsqueeze(0))

In [27]:
output

tensor([[8466.5273]], device='cuda:0')

In [7]:
from transformers import AutoModelForSequenceClassification
model_new = AutoModelForSequenceClassification.from_pretrained('regression_model/checkpoint-20900/', local_files_only=True)